In [ ]:
import os
os.environ["SQLITE_BUSY_TIMEOUT"] = "30000"  # 30 segundos

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale, scale
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader

In [2]:
data_path= "/home/21953404Victor/Deep/drive-download-20231121T165626Z-001/HotelReservationsPreparedCleanAttributes.csv"
label_path = "/home/21953404Victor/Deep/drive-download-20231121T165626Z-001/HotelReservationsOutput.csv"

In [3]:
features  = pd.read_csv(data_path)
labels = pd.read_csv(label_path)

In [4]:
from sklearn.model_selection import train_test_split
# Divide el conjunto de datos
features_train, features_test, labels_train, labels_test = train_test_split(features.values, labels.values, test_size=0.1, random_state=42)
features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.1, random_state=42)

# Convierte los arrays a tensores de PyTorch
features_train = torch.tensor(features_train).float()
labels_train = torch.tensor(labels_train).float()
features_val = torch.tensor(features_val).float()
labels_val = torch.tensor(labels_val).float()
features_test = torch.tensor(features_test).float()
labels_test = torch.tensor(labels_test).float()

# Crea conjuntos de datos de PyTorch
train_dataset = TensorDataset(features_train, labels_train)
val_dataset = TensorDataset(features_val, labels_val)
test_dataset = TensorDataset(features_test, labels_test)

# Crea DataLoaders de PyTorch
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
import tensorflow as tf
import torch

# Verificar si CUDA está disponible
cuda_available = torch.cuda.is_available()
print(f"CUDA disponible: {cuda_available}")

# Enumerar dispositivos de GPU disponibles y mostrar sus nombres si CUDA está disponible
if cuda_available:
    print("Listado de dispositivos GPU disponibles:")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No se detectó una GPU. Asegúrate de que el acelerador de hardware esté configurado.")


2023-11-29 18:54:24.433788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 18:54:24.564989: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 18:54:25.147831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native:/opt/hadoop/lib/native:
2023-11-29 18:54:25.147887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvin

CUDA disponible: True
Listado de dispositivos GPU disponibles:
GPU 0: NVIDIA GeForce RTX 3080 Ti


In [6]:
import torch.nn as nn
import random

class PyTorchModel(nn.Module):
    def __init__(self, trial, input_shape):
        super(PyTorchModel, self).__init__()
        self.layers = nn.ModuleList()
        self.l2_regs = []
        current_shape = input_shape

        # Diccionario para rastrear capas elegibles para conexiones residuales, organizadas por tamaño
        potential_residuals = {size: [] for size in [16, 32, 64, 128, 256, 512, 1024, 2048]}
        self.residual_connections = {}

        n_layers = trial.suggest_int('n_layers', 1, 100)
        
        # Optimizar el umbral para conexiones residuales
        residual_threshold_factor = trial.suggest_float('residual_threshold_factor', 0.5, 1.0)
        residual_treshold = int(n_layers * residual_threshold_factor)
        
        for i in range(n_layers):
            num_units = trial.suggest_categorical(f'n_units_l{i}', [16, 32, 64, 128, 256, 512, 1024, 2048])
            activation = trial.suggest_categorical(f'activation_l{i}', ['ReLU', 'Tanh', 'ELU', 'SELU', 'LeakyReLU'])
            dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.0, 0.5)
            use_batch_norm = trial.suggest_categorical(f'batch_norm_l{i}', [True, False])
            l2_reg = trial.suggest_float(f'l2_reg_l{i}', 1e-5, 1e-1, log=True)

            # Crear y añadir la capa lineal
            layer = nn.Linear(current_shape, num_units)
            self.layers.append(layer)
            self.l2_regs.append(l2_reg)
            
            if num_units in potential_residuals and i >= residual_treshold:
                potential_residuals[num_units].append(i)

            # Añadir Batch Normalization y Dropout
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(num_units))
            self.layers.append(nn.Dropout(dropout_rate))

            # Añadir la activación
            if activation == 'LeakyReLU':
                negative_slope = trial.suggest_float(f'leakyrelu_alpha_l{i}', 0.01, 0.3)
                self.layers.append(nn.LeakyReLU(negative_slope=negative_slope))
            else:
                self.layers.append(getattr(nn, activation)())

            current_shape = num_units

            # Configurar conexiones residuales
            if i >= residual_treshold and num_units in potential_residuals:
                res_index = trial.suggest_int(f'res_connection_index_l{i}', 0, len(potential_residuals[num_units]) - 1)
                self.residual_connections[i] = potential_residuals[num_units][res_index]

        # Capa de salida
        self.output = nn.Linear(current_shape, 1)

    def forward(self, x):
        layer_outputs = {}
        for i, layer in enumerate(self.layers):
            x = layer(x)
            layer_outputs[i] = x.clone()
            if i in self.residual_connections:
                res_layer = self.residual_connections[i]
                if layer_outputs[res_layer].size(1) == x.size(1):
                    x = x + layer_outputs[res_layer]
        return torch.sigmoid(self.output(x))

    def get_l2_loss(self):
        l2_loss = 0.0
        for layer, l2_reg in zip(self.layers, self.l2_regs):
            if isinstance(layer, nn.Linear):
                l2_loss += l2_reg * layer.weight.norm(2)
        return l2_loss


Cada capa tiene una lista de posibles capas anteriores con las que puede tener una conexión residual. Para redes con más de residual_treshold capas, el optimizador decide si se realiza una conexión residual y con qué capa anterior, limitando el número máximo de conexiones residuales a 10. La función forward se ha modificado para manejar estas conexiones residuales

In [7]:
from torch.optim import Adam, SGD, RMSprop

def build_model(params, input_shape, is_trial=True):
    # Crear una instancia del modelo con la arquitectura definida
    model = PyTorchModel(params, input_shape)

    if is_trial:
        # Configuración durante la optimización con Optuna
        optimizer_name = params.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
        lr = params.suggest_float('lr', 1e-5, 1e-1, log=True)
        if optimizer_name == 'Adam':
            beta1 = params.suggest_float('adam_beta1', 0.85, 0.99)
            beta2 = params.suggest_float('adam_beta2', 0.9, 0.999)
            optimizer = Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
        elif optimizer_name == 'SGD':
            momentum = params.suggest_float('sgd_momentum', 0.1, 0.9)
            optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
        elif optimizer_name == 'RMSprop':
            alpha = params.suggest_float('rmsprop_alpha', 0.85, 0.99)
            optimizer = RMSprop(model.parameters(), lr=lr, alpha=alpha)
    else:
        # Configuración con parámetros fijos
        optimizer_name = params['optimizer']
        if optimizer_name == 'Adam':
            optimizer = Adam(model.parameters(), lr=params['lr'], betas=(params['adam_beta1'], params['adam_beta2']))
        elif optimizer_name == 'SGD':
            optimizer = SGD(model.parameters(), lr=params['lr'], momentum=params.get('sgd_momentum', 0.9))
        elif optimizer_name == 'RMSprop':
            optimizer = RMSprop(model.parameters(), lr=params['lr'], alpha=params.get('rmsprop_alpha', 0.99))

    return model, optimizer


In [8]:
from tqdm import tqdm
import torch

def calculate_accuracy(outputs, labels):
    predicted = outputs.round()  # Redondear para obtener las predicciones binarias
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)

In [9]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import gc
from torch.utils.data import DataLoader

def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, file_name_base):
    plt.figure(figsize=(12, 4))

    # Pérdida
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    # Precisión
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    plt.savefig(f"{file_name_base}_plot.png")
    plt.close()

In [10]:
import json

best_accuracy_file = "global_best_accuracy_residual.json"

# Intenta cargar la mejor precisión global desde un archivo
try:
    with open(best_accuracy_file, "r") as file:
        global_best_accuracy = json.load(file)
except (FileNotFoundError, json.JSONDecodeError, TypeError, ValueError):
    global_best_accuracy = 0.0
    
best_model_path = "best_model_residual.pt"
plot_file_name_base = "best_model_residual"

print(f"Mejor accuracy global: {global_best_accuracy}")

Mejor accuracy global: 0.0


In [11]:
import torch
import gc
from torch.utils.data import DataLoader

def objective(trial):
    global global_best_accuracy, best_model_path, plot_file_name_base
    # Verificar y configurar el uso de la GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando {'GPU' if device.type == 'cuda' else 'CPU'} para el Trial {trial.number}")

    model, optimizer = build_model(trial, input_shape)  # Asegúrate de que 'input_shape' esté definido
    model.to(device)

    # Parametrización del número de épocas y tamaño del batch
    epochs = trial.suggest_int('epochs', 10, 2000)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024, 2048])

    # Criterio de pérdida y DataLoader
    criterion = torch.nn.BCELoss()  # Ajustar según tu problema
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Configuración de Early Stopping y Learning Rate Scheduler
    early_stopping_patience = trial.suggest_int('early_stopping_patience', 3, 20)
    factor = trial.suggest_float("lr_reduce_factor", 0.1, 0.9)
    lr_patience = trial.suggest_int("lr_reduce_patience", 5, 50)
    min_lr = trial.suggest_float("min_lr", 1e-6, 1e-4, log=True)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, patience=lr_patience, min_lr=min_lr, verbose=True)

    # Entrenamiento del modelo
    print(f"Entrenando Trial {trial.number}...")
    best_accuracy = 0.0  # Inicializacion de la mejor precisión
    epochs_no_improve = 0
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
            train_total += labels.size(0)

        train_loss /= train_total
        train_accuracy = train_correct / train_total

        # Almacenar las métricas de entrenamiento
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Evaluación y Early Stopping
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item() * inputs.size(0)
                val_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
                val_total += labels.size(0)

        val_loss /= val_total
        val_accuracy = val_correct / val_total

        # Almacenar las métricas de validación
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        lr_scheduler.step(val_loss)

        # Comprobar si este modelo es el mejor hasta ahora
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            if best_accuracy > global_best_accuracy:
                global_best_accuracy = best_accuracy
                # Guardar el nuevo valor de global_best_accuracy
                with open(best_accuracy_file, "w") as file:
                    json.dump(global_best_accuracy, file)
                torch.save(model.state_dict(), best_model_path)  # Sobreescribir el mejor modelo
                plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, plot_file_name_base)  # Sobreescribir el gráfico
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == early_stopping_patience:
            print("Early stopping")
            break
            
    print(f"Trial {trial.number} completado. Mejor precisión en validación: {best_accuracy}")

    # Limpiar la memoria después de cada trial
    del model, optimizer
    torch.cuda.empty_cache()
    gc.collect()

    # Devolver la pérdida en lugar de la precisión
    return best_accuracy


In [12]:
input_shape = features_train.shape[1]  # Número de variables explicativas

In [13]:
import optuna

storage = "sqlite:///example.db"  # Ruta de la base de datos SQLite
study_name = "residual_accuracy_gpu_study3"  # Nuevo nombre del estudio

# Crear un nuevo estudio
study = optuna.create_study(study_name=study_name, direction='maximize', storage=storage, load_if_exists=True)

# Número de ensayos en el estudio
num_trials = len(study.trials)
print(f"El estudio contiene {num_trials} ensayos.")

/home/21953404Victor/SDC3/tutorial-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-29 18:54:25,937] Using an existing study with name 'residual_accuracy_gpu_study3' instead of creating a new one.


El estudio contiene 5 ensayos.


In [ ]:
trials_restantes=50000-num_trials
study.optimize(objective, n_trials=trials_restantes)

Usando GPU para el Trial 5
Entrenando Trial 5...


[I 2023-11-29 19:05:44,391] Trial 5 finished with value: 0.678407350689127 and parameters: {'n_layers': 94, 'residual_threshold_factor': 0.6277050550203718, 'n_units_l0': 2048, 'activation_l0': 'ELU', 'dropout_l0': 0.468437475662215, 'batch_norm_l0': False, 'l2_reg_l0': 1.4071535986427523e-05, 'n_units_l1': 64, 'activation_l1': 'SELU', 'dropout_l1': 0.09266980996066476, 'batch_norm_l1': False, 'l2_reg_l1': 0.0006814707419421719, 'n_units_l2': 1024, 'activation_l2': 'Tanh', 'dropout_l2': 0.2680998086820162, 'batch_norm_l2': False, 'l2_reg_l2': 0.00015654454352493364, 'n_units_l3': 1024, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.11329911009904509, 'batch_norm_l3': False, 'l2_reg_l3': 5.070870739218057e-05, 'leakyrelu_alpha_l3': 0.12371094142600719, 'n_units_l4': 1024, 'activation_l4': 'Tanh', 'dropout_l4': 0.3747644479586307, 'batch_norm_l4': True, 'l2_reg_l4': 0.04709541261334058, 'n_units_l5': 16, 'activation_l5': 'ReLU', 'dropout_l5': 0.2162120272290563, 'batch_norm_l5': True, 'l2

Early stopping
Trial 5 completado. Mejor precisión en validación: 0.678407350689127
Usando GPU para el Trial 6
Entrenando Trial 6...


Early stopping
Trial 6 completado. Mejor precisión en validación: 0.8196018376722818


[I 2023-11-29 19:25:07,455] Trial 6 finished with value: 0.8196018376722818 and parameters: {'n_layers': 27, 'residual_threshold_factor': 0.8611776313633042, 'n_units_l0': 64, 'activation_l0': 'SELU', 'dropout_l0': 0.4661491088198367, 'batch_norm_l0': False, 'l2_reg_l0': 0.025293599008377243, 'n_units_l1': 128, 'activation_l1': 'Tanh', 'dropout_l1': 0.30329813751135676, 'batch_norm_l1': True, 'l2_reg_l1': 0.026018871380650836, 'n_units_l2': 32, 'activation_l2': 'LeakyReLU', 'dropout_l2': 0.41780700470589327, 'batch_norm_l2': True, 'l2_reg_l2': 2.563295657603275e-05, 'leakyrelu_alpha_l2': 0.22810490919654333, 'n_units_l3': 256, 'activation_l3': 'Tanh', 'dropout_l3': 0.24614477054655526, 'batch_norm_l3': False, 'l2_reg_l3': 0.01629291288268337, 'n_units_l4': 32, 'activation_l4': 'ReLU', 'dropout_l4': 0.25916458485189664, 'batch_norm_l4': False, 'l2_reg_l4': 0.03125781558298025, 'n_units_l5': 1024, 'activation_l5': 'ReLU', 'dropout_l5': 0.1742736023626078, 'batch_norm_l5': True, 'l2_reg_l

...

[I 2023-11-29 20:50:07,818] Trial 42 finished with value: 0.8361408882082695 and parameters: {'n_layers': 27, 'residual_threshold_factor': 0.8954151454388746, 'n_units_l0': 16, 'activation_l0': 'ELU', 'dropout_l0': 0.14641034780945056, 'batch_norm_l0': True, 'l2_reg_l0': 0.0015395981095616272, 'n_units_l1': 128, 'activation_l1': 'SELU', 'dropout_l1': 0.07153726169520418, 'batch_norm_l1': True, 'l2_reg_l1': 6.414928099917724e-05, 'n_units_l2': 1024, 'activation_l2': 'SELU', 'dropout_l2': 0.42054535349362515, 'batch_norm_l2': False, 'l2_reg_l2': 0.00029655273586483077, 'n_units_l3': 256, 'activation_l3': 'ReLU', 'dropout_l3': 0.17067608149405278, 'batch_norm_l3': False, 'l2_reg_l3': 0.016703740737605152, 'n_units_l4': 128, 'activation_l4': 'ELU', 'dropout_l4': 0.3925536007812939, 'batch_norm_l4': True, 'l2_reg_l4': 0.04780954365564726, 'n_units_l5': 1024, 'activation_l5': 'SELU', 'dropout_l5': 0.22116020205609765, 'batch_norm_l5': True, 'l2_reg_l5': 0.0970495273815794, 'n_units_l6': 64, 

Usando GPU para el Trial 43
Entrenando Trial 43...


[I 2023-11-29 20:52:22,998] Trial 43 finished with value: 0.678407350689127 and parameters: {'n_layers': 83, 'residual_threshold_factor': 0.9557812721970955, 'n_units_l0': 256, 'activation_l0': 'Tanh', 'dropout_l0': 0.05853752753728647, 'batch_norm_l0': False, 'l2_reg_l0': 0.0005500604395797707, 'n_units_l1': 128, 'activation_l1': 'Tanh', 'dropout_l1': 0.13555499229357093, 'batch_norm_l1': True, 'l2_reg_l1': 0.00015160781129461748, 'n_units_l2': 16, 'activation_l2': 'ReLU', 'dropout_l2': 0.33449759689606284, 'batch_norm_l2': False, 'l2_reg_l2': 0.003015357434175578, 'n_units_l3': 1024, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.08855528378147193, 'batch_norm_l3': True, 'l2_reg_l3': 0.0987179193775214, 'leakyrelu_alpha_l3': 0.29079941809237514, 'n_units_l4': 1024, 'activation_l4': 'SELU', 'dropout_l4': 0.4669640680057173, 'batch_norm_l4': False, 'l2_reg_l4': 0.014493874919378244, 'n_units_l5': 64, 'activation_l5': 'Tanh', 'dropout_l5': 0.305355292706156, 'batch_norm_l5': True, 'l2_re

Early stopping
Trial 43 completado. Mejor precisión en validación: 0.678407350689127
Usando GPU para el Trial 44
Entrenando Trial 44...


Epoch 33/1740:  30%|████████████████████████████████████████████████████████▊                                                                                                                                       | 544/1837 [00:03<00:08, 153.00it/s]

In [ ]:
print('Número de pruebas finalizadas:', len(study.trials))
print('Mejor prueba:', study.best_trial.params)

## Test

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

best_params = study.best_trial.params

# Crear el modelo, el optimizador y el scheduler usando la función build_model
model, optimizer = build_model(best_params, input_shape, is_trial=False)

# Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Criterio de pérdida
criterion = nn.BCELoss()  # Ajusta según tu caso

# Función para calcular la precisión
def calculate_accuracy(outputs, labels):
    predicted = outputs.round()  # Redondear para obtener las predicciones binarias
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)

# Variables para almacenar métricas y manejar el early stopping
best_accuracy = 0.0
epochs_no_improve = 0
early_stopping_patience = best_params['early_stopping_patience']

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
# Bucle de entrenamiento
epochs = best_params['epochs']
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
        train_total += labels.size(0)

    train_loss /= train_total
    train_accuracy = train_correct / train_total

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Evaluación y Early Stopping
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item() * inputs.size(0)
            val_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
            val_total += labels.size(0)

    val_loss /= val_total
    val_accuracy = val_correct / val_total

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    lr_scheduler.step(val_loss)

    print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Descomentar si deseas aplicar early stopping
    # if epochs_no_improve == early_stopping_patience:
    #     print("Early stopping")
    #     break

print(f"Entrenamiento completado. Mejor precisión en validación: {best_accuracy}")

In [ ]:
# Graficar la pérdida y la precisión
plt.figure(figsize=(12, 4))

# Pérdida
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# Precisión
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()

plt.show()

In [ ]:
# Evaluar el modelo en el conjunto de prueba
model.evaluate(features_test, labels_test)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# Asegurarse de que el modelo esté en modo de evaluación y moverlo al dispositivo adecuado
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Realizar las predicciones
with torch.no_grad():
    inputs = features_test.to(device)
    outputs = model(inputs)
    predictions = outputs.round().cpu().numpy()  # Redondear las predicciones y moverlas al CPU

# Calcular y visualizar la matriz de confusión
conf_matrix = confusion_matrix(labels_test.numpy(), predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.show()

# Calcular y mostrar otras métricas
accuracy = accuracy_score(labels_test.numpy(), predictions)
recall = recall_score(labels_test.numpy(), predictions)
precision = precision_score(labels_test.numpy(), predictions)
f1 = f1_score(labels_test.numpy(), predictions)
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
